In [43]:
import os, sys

base_path = os.path.join(os.getcwd(), "..")
print(f"Base Path: {base_path}")
sys.path.append(base_path)

Base Path: /Users/swkim/Documents/coding/thesis/PROMES_colab/notebook/..


In [44]:
import gym
from kube_sim_gym.envs.sim_kube_env import SimKubeEnv

from stable_baselines3 import DQN, PPO
import torch

In [45]:
sample1 = torch.tensor([[0.99, 0.90, 0.80, 0.80, 0.95, 0.95, 0.90, 0.85, 0.0, 0.0, 0.0, 0.0]])
sample2 = torch.tensor([[0.99, 0.90, 0.80, 0.80, 0.95, 0.95, 0.90, 0.85, 0.0, 0.0, 0.6, 0.7]])
sample3 = torch.tensor([[0.99, 0.90, 0.40, 0.40, 0.15, 0.15, 0.90, 0.85, 0.8, 0.8, 0.6, 0.7]])

In [46]:
env = gym.make('SimKubeEnv-v0', reward_file='promes_static.py', scenario_file='scenario-5l-5m-1000p-10m.csv')
model = PPO.load('training/model/PPO_Promes_Dynamic_New', env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [47]:
model_policy = model.policy
print(model_policy)

ActorCriticPolicy(
  (features_extractor): PromesPPO(
    (net): Net5_(
      (net3_): Net3_(
        (fc1_1): Linear(in_features=10, out_features=16, bias=True)
        (fc1_2): Linear(in_features=2, out_features=16, bias=True)
        (fc2): Linear(in_features=32, out_features=16, bias=True)
        (fc3): Linear(in_features=16, out_features=16, bias=True)
        (fc4): None
      )
      (fc1_3_1): Linear(in_features=2, out_features=16, bias=True)
      (fc1_3_2): Linear(in_features=2, out_features=16, bias=True)
      (fc1_3_3): Linear(in_features=2, out_features=16, bias=True)
      (fc1_3_4): Linear(in_features=2, out_features=16, bias=True)
      (fc1_3_5): Linear(in_features=2, out_features=16, bias=True)
      (fc2_1): Linear(in_features=32, out_features=16, bias=True)
      (fc3_1): None
      (fc2_2): Linear(in_features=32, out_features=16, bias=True)
      (fc3_2): None
      (fc2_3): Linear(in_features=32, out_features=16, bias=True)
      (fc3_3): None
      (fc2_4): Lin

In [48]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F

from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from gym import spaces

from kube_mm_scheduler.model.promes import Net5_

device = th.device("cuda" if th.cuda.is_available() else "cpu")

class PromesPPO(BaseFeaturesExtractor):
    def __init__(self, observation_space: spaces.Box, features_dim: int = 80):
        super(PromesPPO, self).__init__(observation_space, features_dim)
        self.net = Net5_().to(device)
        self.net.load_state_dict(th.load(os.path.join(base_path,'kube_mm_scheduler/weight/net5.pt')))
        self.net.eval()

    def forward(self, observations: th.Tensor) -> th.Tensor:
        input1 = observations[:, :10].to(device)
        input2 = observations[:, 10:].to(device)

        return self.net(input1, input2)

policy_kwargs = dict(
    features_extractor_class=PromesPPO,
    features_extractor_kwargs=dict(features_dim=80),
)

model = PPO('MlpPolicy', env, verbose=1, policy_kwargs=policy_kwargs)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [111]:
features = model.policy.extract_features(sample1)
features

tensor([[-6.8209e-01, -2.0166e-03,  4.3477e-02, -4.1246e-01,  8.4649e-01,
          1.2364e+00,  3.6112e-01, -7.0216e-02, -1.2488e+00, -8.4166e-01,
         -6.3948e-01,  3.7009e-01,  9.2319e-01, -1.1981e+00,  2.1128e-01,
         -6.8118e-01, -8.1349e-01,  6.4176e-01,  9.6026e-02, -9.1739e-01,
          7.6262e-01, -3.5616e-01, -4.7152e-01, -3.4970e-01,  4.7748e-01,
         -2.9206e-01, -6.0026e-01, -5.0589e-01,  1.6936e-01, -3.5726e-01,
          2.2118e-01,  1.6680e-01,  1.0720e+00, -3.8339e-01,  2.0188e-01,
          2.6974e-01,  1.0602e+00,  2.7954e-01, -3.0595e-01,  4.4118e-01,
         -5.7934e-01, -1.1713e-01, -9.1546e-01, -3.5743e-01, -1.9049e-01,
         -6.7302e-01, -1.3156e-01, -1.0659e+00,  5.0722e-01, -1.1593e+00,
         -1.9281e+00,  9.0136e-01, -2.9678e-01, -1.1226e+00, -8.9587e-01,
         -3.8955e-01,  2.4844e-01,  5.0076e-02, -6.8017e-01, -3.4872e-01,
          5.4871e-01, -3.4983e-01,  8.3166e-02, -1.9030e-01,  9.8957e-01,
         -5.7375e-02,  1.4242e-01,  4.

In [112]:
latent_pi, latent_vf = model.policy.mlp_extractor(features)
latent_pi

tensor([[-0.3412, -0.9104, -0.5699, -0.8910,  0.5330,  0.9437,  0.7276, -0.2277,
         -0.4597, -0.2187,  0.7416, -0.1159, -0.4737, -0.2007,  0.3745,  0.6154,
          0.6743, -0.9535,  0.2167,  0.1879,  0.9270, -0.5601,  0.0476, -0.1895,
         -0.7922,  0.7731,  0.5519, -0.4477,  0.4764, -0.2538, -0.5635,  0.6201,
          0.6768,  0.7503, -0.8062, -0.6625,  0.3031,  0.9158, -0.8296,  0.6666,
          0.7554, -0.7696, -0.1408,  0.6410,  0.1657,  0.8620, -0.3111, -0.9673,
         -0.4229,  0.2294,  0.8908,  0.3504, -0.6294,  0.8077,  0.2003, -0.9478,
          0.0798,  0.0733,  0.6702,  0.3428,  0.5562, -0.8597, -0.3336,  0.0386]],
       grad_fn=<TanhBackward0>)

In [117]:
scores = model.policy.action_net(latent_pi)
scores

tensor([[ 0.0016, -0.0082,  0.0031, -0.0055, -0.0064, -0.0100]],
       grad_fn=<AddmmBackward0>)

In [119]:
scores.tolist()[0]

[0.001594634260982275,
 -0.008207956328988075,
 0.0030819326639175415,
 -0.005500008352100849,
 -0.00635987613350153,
 -0.009973096661269665]

In [116]:
model.policy.predict(sample1)

(array([2]), None)

In [113]:
cd = model.policy._get_action_dist_from_latent(latent_pi)
cd

In [110]:
cd.log_prob([1,2,3,4,5,6])

ValueError: The value argument to log_prob must be a Tensor

In [38]:
# From stable_baselines3 model, exclude the last layer
import stable_baselines3 as sb3

observation_space = env.observation_space
action_space = env.action_space


sb3.common.policies.ActorCriticPolicy()

TypeError: __init__() missing 3 required positional arguments: 'observation_space', 'action_space', and 'lr_schedule'

In [36]:
model.policy.features_extractor

tensor([[-0.0747, -0.5190,  0.0521, -0.8373,  0.9420,  0.9265, -0.2214,  0.0048,
         -0.3006, -0.9299, -0.6604, -0.0232,  0.2931,  0.0276,  0.0727,  0.5788,
         -0.2546,  0.7006,  0.2385,  0.3027, -0.4275,  0.2718,  0.5836, -0.2001,
          0.1188, -0.4523,  0.2804, -0.5034, -0.2554,  0.8030, -0.4536, -0.1466,
          0.1496, -0.2085, -0.0995, -0.2559, -0.1075,  0.5839, -0.7997,  0.2909,
         -0.6887, -0.4395, -0.1927,  0.2446, -0.6809, -0.0246,  0.2542, -0.3372,
          0.5050, -0.0510, -1.2866, -0.4210, -0.4491, -0.1291, -0.5756,  0.1960,
          0.2474, -0.6453,  0.2328, -0.1804,  0.0487,  0.1516, -0.1517, -0.1941,
          0.0721,  0.1695,  0.3108,  0.0065, -0.1304, -0.2700, -0.0146, -0.0317,
         -0.7572,  0.1292,  0.4665, -0.1370, -0.4350,  0.0142, -0.1797, -0.1488]],
       grad_fn=<CatBackward0>)

In [16]:
print(model_pytorch)

ActorCriticPolicy(
  (features_extractor): PromesPPO(
    (net): Net5_(
      (net3_): Net3_(
        (fc1_1): Linear(in_features=10, out_features=16, bias=True)
        (fc1_2): Linear(in_features=2, out_features=16, bias=True)
        (fc2): Linear(in_features=32, out_features=16, bias=True)
        (fc3): Linear(in_features=16, out_features=16, bias=True)
        (fc4): None
      )
      (fc1_3_1): Linear(in_features=2, out_features=16, bias=True)
      (fc1_3_2): Linear(in_features=2, out_features=16, bias=True)
      (fc1_3_3): Linear(in_features=2, out_features=16, bias=True)
      (fc1_3_4): Linear(in_features=2, out_features=16, bias=True)
      (fc1_3_5): Linear(in_features=2, out_features=16, bias=True)
      (fc2_1): Linear(in_features=32, out_features=16, bias=True)
      (fc3_1): None
      (fc2_2): Linear(in_features=32, out_features=16, bias=True)
      (fc3_2): None
      (fc2_3): Linear(in_features=32, out_features=16, bias=True)
      (fc3_3): None
      (fc2_4): Lin

In [ ]:
model.fea

In [126]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from gym import spaces

from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.logger import configure

from kube_mm_scheduler.model.promes import Net5_
from kube_mm_scheduler.model.net3 import Model as Net3

from typing import Dict, List, Tuple, Type, Union


device = th.device("cuda" if th.cuda.is_available() else "cpu")


class Net3_(Net3):
    def __init__(self):
        super(Net3_, self).__init__()
        self.fc4 = None

    def forward(self, x1, x2):
        x1 = F.relu(self.fc1_1(x1))  
        x2 = F.relu(self.fc1_2(x2))
        x = torch.cat((x1, x2), dim=1) 
        x = F.relu(self.fc2(x))  
        x = F.relu(self.fc3(x))
        return x


class PromesPPO(BaseFeaturesExtractor):
    def __init__(self, observation_space: spaces.Box, features_dim: int = 80):
        super(PromesPPO, self).__init__(observation_space, features_dim)
        self.net = Net5_().to(device)
        self.net.load_state_dict(th.load(os.path.join(base_path,'kube_mm_scheduler/weight/net5.pt')))
        self.net.eval()

    def forward(self, observations: th.Tensor) -> th.Tensor:
        input1 = observations[:, :10].to(device)
        input2 = observations[:, 10:].to(device)

        return self.net(input1, input2)

policy_kwargs = dict(
    features_extractor_class=PromesPPO,
    features_extractor_kwargs=dict(features_dim=80),
)

class MlpExtractor(nn.Module):

    def __init__(
        self,
        feature_dim: int,
        net_arch: Union[List[int], Dict[str, List[int]]],
        activation_fn: Type[nn.Module],
        device: Union[th.device, str] = "auto",
    ) -> None:
        super().__init__()
        policy_net: List[nn.Module] = []
        value_net: List[nn.Module] = []
        last_layer_dim_pi = feature_dim
        last_layer_dim_vf = feature_dim

        # save dimensions of layers in policy and value nets
        if isinstance(net_arch, dict):
            # Note: if key is not specificed, assume linear network
            pi_layers_dims = net_arch.get("pi", [])  # Layer sizes of the policy network
            vf_layers_dims = net_arch.get("vf", [])  # Layer sizes of the value network
        else:
            pi_layers_dims = vf_layers_dims = net_arch
        # Iterate through the policy layers and build the policy net
        for curr_layer_dim in pi_layers_dims:
            policy_net.append(nn.Linear(last_layer_dim_pi, curr_layer_dim))
            policy_net.append(activation_fn())
            last_layer_dim_pi = curr_layer_dim
        # Iterate through the value layers and build the value net
        for curr_layer_dim in vf_layers_dims:
            value_net.append(nn.Linear(last_layer_dim_vf, curr_layer_dim))
            value_net.append(activation_fn())
            last_layer_dim_vf = curr_layer_dim

        # Save dim, used to create the distributions
        self.latent_dim_pi = last_layer_dim_pi
        self.latent_dim_vf = last_layer_dim_vf

        # Create networks
        # If the list of layers is empty, the network will just act as an Identity module
        self.policy_net = nn.Sequential(*policy_net).to(device)
        self.value_net = nn.Sequential(*value_net).to(device)

    def forward(self, features: th.Tensor) -> Tuple[th.Tensor, th.Tensor]:
        """
        :return: latent_policy, latent_value of the specified network.
            If all layers are shared, then ``latent_policy == latent_value``
        """
        return self.forward_actor(features), self.forward_critic(features)

    def forward_actor(self, features: th.Tensor) -> th.Tensor:
        return self.policy_net(features)

    def forward_critic(self, features: th.Tensor) -> th.Tensor:
        return self.value_net(features)

In [127]:
import torch
import torch.nn as nn

class ActorCriticPolicy(nn.Module):

    def __init__(self):
        super().__init__()

        # Features extractor
        self.features_extractor = PromesPPO(observation_space=env.observation_space ,features_dim=80)

        # MLP extractor
        self.mlp_extractor = MlpExtractor(feature_dim=80, net_arch=[64, 64], activation_fn=nn.Tanh, device=device)

        # Action net
        self.action_net = nn.Linear(in_features=64, out_features=6, bias=True)

        # Value net
        self.value_net = nn.Linear(in_features=64, out_features=1, bias=True)

    def forward(self, state):
        # Extract features
        features = self.features_extractor(state)

        # Extract policy and value
        policy = self.mlp_extractor.policy_net(features)
        value = self.mlp_extractor.value_net(features)

        # Get action
        action = self.action_net(policy)

        # Get value
        value = self.value_net(value)

        return action, value

In [128]:
# Make a model with the same parameters
pt_model = ActorCriticPolicy()

In [129]:
# Extract weights from model_policy
weights = model_policy.state_dict()

In [130]:
# Load weights to pt_model
pt_model.load_state_dict(weights)

<All keys matched successfully>

In [131]:
sample1 = torch.tensor([[0.99, 0.90, 0.80, 0.80, 0.95, 0.95, 0.90, 0.85, 0.0, 0.0, 0.0, 0.0]])
sample2 = torch.tensor([[0.99, 0.90, 0.80, 0.80, 0.95, 0.95, 0.90, 0.85, 0.0, 0.0, 0.6, 0.7]])
sample3 = torch.tensor([[0.99, 0.90, 0.40, 0.40, 0.15, 0.15, 0.90, 0.85, 0.8, 0.8, 0.6, 0.7]])

In [123]:
pt_model(sample1)

(tensor([[-0.0138,  0.0029,  0.0015, -0.0032,  0.0009,  0.0082]],
        grad_fn=<AddmmBackward0>),
 tensor([[-0.2822]], grad_fn=<AddmmBackward0>))

In [98]:
pt_model(sample1)[0]

tensor([[-6.2938,  1.9472, -0.3218,  0.2534,  2.2748, -3.2760]],
       grad_fn=<AddmmBackward0>)

In [81]:
pt_model(sample2)

(tensor([[-5.9904, -0.0553, -0.9415, -1.1002, -0.3881,  4.7698]],
        grad_fn=<AddmmBackward0>),
 tensor([[-61.6606]], grad_fn=<AddmmBackward0>))

In [83]:
pt_model(sample3)

(tensor([[-6.6532, -0.6983,  1.0152,  0.3938,  1.4332, -0.1519]],
        grad_fn=<AddmmBackward0>),
 tensor([[5.3250]], grad_fn=<AddmmBackward0>))